In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
import numpy as np
import re

def height_map(x):
	if x is np.nan:
		return np.nan
	
	feet, inches = map(int, x.split('-'))
	return (feet * 12) + inches

def round_map(round):
	if round is np.nan:
		return np.nan
	
	roundSection = re.search("\/(.*?)\/", round)
	round = re.search("([0-9])", roundSection[0]) 

	return int(round[0])

def day_map(round):
	if round is np.nan:
		return np.nan
	
	if round == 1:
		return 1
	
	if round == 2 or round == 3:
		return 2
	
	return 3

# print(height_map("5-11"))

data = pd.read_csv("./combine-data.csv")

data['Ht'] = data['Ht'].map(height_map)
data['Round'] = data['Drafted (tm/rnd/yr)'].map(round_map)
data['Day'] = data['Round'].map(day_map)

data = data.drop(['College', 'Drafted (tm/rnd/yr)', 'Player-additional'], axis=1)

data.head()
# TODO: add pick number


<>:21: SyntaxWarning: invalid escape sequence '\/'
<>:21: SyntaxWarning: invalid escape sequence '\/'
C:\Users\benja\AppData\Local\Temp\ipykernel_41168\392341119.py:21: SyntaxWarning: invalid escape sequence '\/'
  roundSection = re.search("\/(.*?)\/", round)


,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Round,Day
0,Kris Abrams-Draine,CB,Missouri,71.0,179.0,4.44,33.5,NaN,NaN,NaN,NaN,5.0,3
1,Isaiah Adams,G,Illinois,76.0,315.0,5.22,24.5,NaN,102.0,7.77,4.73,3.0,2
2,Rasheen Ali,RB,Marshall,71.0,206.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3
3,Erick All,TE,Iowa,76.0,252.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3
4,Braelon Allen,RB,Wisconsin,73.0,235.0,NaN,32.0,26.0,117.0,NaN,NaN,4.0,3


In [149]:
features = ["Pos","School", "Ht", "Wt", "40yd", "Vertical", "Bench", "Broad Jump", "3Cone", "Shuttle"]

X, y = data[features], data['Day']

print(X.shape)

(8320, 10)


In [150]:
# Update all nan to be the mean

for f in features:
	if (f == 'Pos' or f == 'School'):
		continue

	X[f] = X[f].fillna((X[f].mean()))


# X = X.fillna(0)
y = y.fillna(8)

X

C:\Users\benja\AppData\Local\Temp\ipykernel_23752\682369739.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[f] = X[f].fillna((X[f].mean()))


,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
0,CB,Missouri,71.0,179.0,4.440000,33.500000,20.72936,114.886969,7.284998,4.40161
1,G,Illinois,76.0,315.0,5.220000,24.500000,20.72936,102.000000,7.770000,4.73000
2,RB,Marshall,71.0,206.0,4.772474,32.967897,20.72936,114.886969,7.284998,4.40161
3,TE,Iowa,76.0,252.0,4.772474,32.967897,20.72936,114.886969,7.284998,4.40161
4,RB,Wisconsin,73.0,235.0,4.772474,32.000000,26.00000,117.000000,7.284998,4.40161
...,...,...,...,...,...,...,...,...,...,...
8315,DT,Texas,74.0,290.0,5.210000,30.500000,23.00000,109.000000,7.284998,4.40161
8316,RB,Jackson State,71.0,216.0,4.840000,29.500000,15.00000,112.000000,7.150000,4.41000
8317,QB,Texas State,75.0,229.0,4.910000,34.000000,20.72936,108.000000,7.710000,4.59000
8318,WR,Iowa,75.0,191.0,4.530000,33.000000,20.72936,130.000000,7.090000,4.18000


In [151]:
# Encode Position

X.Pos=X.Pos.astype('category').cat.codes
X.School=X.School.astype('category').cat.codes

print(X.head())

   Pos  School    Ht     Wt      40yd   Vertical     Bench  Broad Jump  \
0    1     180  71.0  179.0  4.440000  33.500000  20.72936  114.886969   
1    8     124  76.0  315.0  5.220000  24.500000  20.72936  102.000000   
2   19     158  71.0  206.0  4.772474  32.967897  20.72936  114.886969   
3   22     128  76.0  252.0  4.772474  32.967897  20.72936  114.886969   
4   19     348  73.0  235.0  4.772474  32.000000  26.00000  117.000000   

      3Cone  Shuttle  
0  7.284998  4.40161  
1  7.770000  4.73000  
2  7.284998  4.40161  
3  7.284998  4.40161  
4  7.284998  4.40161  


C:\Users\benja\AppData\Local\Temp\ipykernel_23752\3637210413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Pos=X.Pos.astype('category').cat.codes
C:\Users\benja\AppData\Local\Temp\ipykernel_23752\3637210413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.School=X.School.astype('category').cat.codes


In [152]:
# MinMaxScaler

feature_scaler = MinMaxScaler()
X = feature_scaler.fit_transform(X)

In [153]:
def train_predict(X_train, y_train, X_test, y_test):
	model =  RandomForestClassifier()
	model.fit(X_train, y_train)

	# 6 Evaluation: Test the model via predicting
	y_pred = model.predict(X_train)
	acc_train = accuracy_score(y_train, y_pred)
	print("accuracy score (training data): %.2f" % acc_train)
	y_pred = model.predict(X_test)
	acc_test = accuracy_score(y_test, y_pred)
	print("0.5 Split score: %.2f" % acc_test)

In [154]:
# Split the data into Train and Test so we can predict
TEST_PROP, RANDOM_SEED = 0.5, 0
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_PROP, random_state=RANDOM_SEED)

In [155]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)

print(kf)

KFold(n_splits=2, random_state=None, shuffle=False)

model =  RandomForestClassifier()

predictions = cross_val_predict(model, X, y, cv=kf)
scores = cross_val_score(model, X, y, cv=kf)

train_predict(X_train, y_train, X_test, y_test)
for index, score in np.ndenumerate(scores):
	print('k-Fold:', index[0], ':', score)

KFold(n_splits=10, random_state=None, shuffle=False)
accuracy score (training data): 1.00
0.5 Split score: 0.70
k-Fold: 0 : 0.6838942307692307
k-Fold: 1 : 0.7415865384615384
k-Fold: 2 : 0.703125
k-Fold: 3 : 0.7007211538461539
k-Fold: 4 : 0.6923076923076923
k-Fold: 5 : 0.6983173076923077
k-Fold: 6 : 0.703125
k-Fold: 7 : 0.7055288461538461
k-Fold: 8 : 0.7103365384615384
k-Fold: 9 : 0.7115384615384616
